In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.datasets.samples_generator import make_blobs
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler , LabelEncoder , OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc
from sklearn.mixture import GaussianMixture as GMM

# DATASET1

In [ ]:
##Importing Data from the file 
df_data=pd.read_csv("C:/Users/PC/Desktop/ML/sgemm_product.csv")
df_data.head()

###Taking average log values for the Last four attributes.
df_data["Avg_run"]=df_data.iloc[:,14:].mean(axis=1)
df_data["log_avg_run"]=np.log(df_data["Avg_run"])
df_data.drop(['Run1 (ms)','Run2 (ms)','Run3 (ms)','Run4 (ms)','Avg_run'],axis=1,inplace=True)
df_data.head()

###Performing binary classification of data.
median=df_data.log_avg_run.median()
median
##Splitting data over median value.
X=df_data.iloc[:,1:14]
Y=df_data.iloc[:,14].apply(lambda x : 0 if x <= median else 1)
Y.value_counts()
###Performing standardization

X_s=X.copy()
standardscalar=StandardScaler()
X_s=standardscalar.fit_transform(X_s)

#########Spliting into Train-Test######################

X_train,X_test, y_train, y_test = train_test_split(X_s,Y,test_size = 0.25 , random_state = 0)



# k-MEANS

In [ ]:
# Using elblow method to find optimal number of clusters
from sklearn.cluster import KMeans
wcss = []
for i in range (1,30):
    kmeans = KMeans(n_clusters=i ,  
                    init = "k-means++" , 
                    max_iter= 500 , 
                    n_init= 10 , 
                    random_state= 0 )
    kmeans.fit(X_s)
    wcss.append(kmeans.inertia_)

plt.plot(range(1,30),wcss,marker='o')
plt.grid()
plt.title("Error Rate-Elbow Method")
plt.xlabel("Number of Cluster")
plt.ylabel("Error")
plt.show()

#####Applying K-means algorithm
kmeans = KMeans(n_clusters= 10, 
                init = "k-means++" , 
                max_iter= 500 , 
                n_init= 10 , 
                random_state= 0)

y_kmeans = kmeans.fit_predict(X_s)

# DIMENTIONALITY REDUCTION-PCA/ICA/RP/FS

In [ ]:
# Implementing PCA for Energy dataset to find maximum variance 

pca = PCA(n_components= None)
X_PCA = pca.fit_transform(X_s)
#X_test = pca.transform(X_test)
explained_variance = pca.explained_variance_ratio_

plt.plot(range(1,14),explained_variance,marker='o')
plt.title("PCA Variance")
plt.xlabel("Component Number")
plt.ylabel("Variance")
plt.grid()
plt.show()

###Choosing Optimal n-components
pca_final_energy = PCA(n_components =10)
X_PCA= pca_final_energy.fit_transform(X_s)

#####Plotting error v/s PCA
wcss_pca = []
for i in range (1,20):
    kmeans = KMeans(n_clusters=i , 
                    init = "k-means++" , 
                    max_iter= 500 , 
                    n_init= 10 , 
                    random_state= 0 )
    kmeans.fit(X_PCA)
    wcss_pca.append(kmeans.inertia_)

plt.plot(range(1,20),wcss_pca,marker='o')
plt.title("PCA-Elbow Method")
plt.grid()
plt.xlabel("Number of Cluster")
plt.ylabel("Error")
plt.show()


In [ ]:
####Implementing ICA
from sklearn.decomposition import FastICA

ica = FastICA(tol = 1e-2, max_iter = 1000, n_components =10)
X_ICA = ica.fit_transform(X_s)

wcss_energy_ica = []
for i in range (1,21):
    kmeans = KMeans(n_clusters=i , 
                    init = "k-means++" , 
                    max_iter= 500 , 
                    n_init= 10 , 
                    random_state= 0 )
    kmeans.fit(X_ICA)
    wcss_energy_ica.append(kmeans.inertia_)

plt.plot(range(1,21),wcss_energy_ica,marker='o')
plt.title("ICA-Elbow Method")
plt.grid()
plt.xlabel("Number of Cluster")
plt.ylabel("Error")
plt.show()


In [ ]:
####Implementing RP

from sklearn.random_projection import SparseRandomProjection
srp = SparseRandomProjection(eps = 0.1, n_components = 10)
X_RP = srp.fit_transform(X_s)

wcss_energy_ra = []
for i in range (1,11):
    kmeans = KMeans(n_clusters=i , 
                    init = "k-means++" , 
                    max_iter= 500 , 
                    n_init= 10 , 
                    random_state= 0 )
    kmeans.fit(X_RP)
    wcss_energy_ra.append(kmeans.inertia_)

plt.plot(range(1,11),wcss_energy_ra,marker='o')
plt.title("Randomized Projection-Elbow Method")
plt.grid()
plt.xlabel("Number of Cluster")
plt.ylabel("Error")
plt.show()

In [ ]:
####Implementing Feature Selection

from sklearn.feature_selection import RFECV
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc

dt = DecisionTreeClassifier(criterion = 'entropy', max_depth=6)
rfecv = RFECV(estimator=dt, step=1, cv=5, n_jobs=-1,
              scoring='accuracy')
rfecv.fit(X,Y)

print("Optimal number of features : %d" % rfecv.n_features_)
print(rfecv.ranking_)

###Plotting Features
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score")
plt.grid()
plt.title("Feature scores")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_,marker='o')
plt.show()

####Plotting error v/s feature

X_FS = X_s[:,(0,2,3,7)]

wcss_FS = []
for i in range (1,20):
    kmeans = KMeans(n_clusters=i , 
                    init = "k-means++" , 
                    max_iter= 500 , 
                    n_init= 10 , 
                    random_state= 0 )
    kmeans.fit(X_FS)
    wcss_FS.append(kmeans.inertia_)

plt.plot(range(1,20),wcss_FS,marker='o')
plt.title("Feature Selection-Elbow Method ")
plt.grid()
plt.xlabel("Number of Cluster")
plt.ylabel("Error")
plt.show()

# IMPLEMENTING ANN USING DIMENTIONALITY REDUCTION

In [ ]:
###ANN CLASSIFIER

#####Final ANN Classifier

ANN_class= MLPClassifier(hidden_layer_sizes= (30,30,30,30),
                          activation='relu',  
                          batch_size='auto' ,
                          solver= 'sgd',
                          learning_rate= 'constant',
                          max_iter= 500,
                          random_state= 0 )
ANN_class.fit(X_train, y_train)
#F
###Predicting
y_pred= ANN_class.predict(X_test)
y_expect = y_test

##Accuracy
acc_dt1 = accuracy_score(y_expect , y_pred)
print(acc_dt1)

### create confusion matrix
cm_dt1= confusion_matrix(y_test , y_pred)
print(cm_dt1)

##ANN Classifier using PCA

X_train,X_test, y_train, y_test = train_test_split(X_PCA,Y,test_size = 0.25 , random_state = 0)

ANN_PCAclass= MLPClassifier(hidden_layer_sizes= (30,30,30,30),
                          activation='relu',  
                          batch_size='auto' ,
                          solver= 'sgd',
                          learning_rate= 'constant',
                          max_iter= 500,
                          random_state= 0 )
ANN_PCAclass.fit(X_train, y_train)

###Predicting
y_pred_pca= ANN_PCAclass.predict(X_test)
y_expect_pca = y_test

##Accuracy
acc_dt1_pca = accuracy_score(y_expect_pca , y_pred_pca)
print(acc_dt1_pca)

### create confusion matrix
cm_dt1_pca= confusion_matrix(y_test , y_pred_pca)
print(cm_dt1_pca)

###ANN Classifier using ICA

X_train,X_test, y_train, y_test = train_test_split(X_ICA,Y,test_size = 0.25 , random_state = 0)

ANN_ICAclass= MLPClassifier(hidden_layer_sizes= (30,30,30,30),
                          activation='relu',  
                          batch_size='auto' ,
                          solver= 'sgd',
                          learning_rate= 'constant',
                          max_iter= 500,
                          random_state= 0 )
ANN_ICAclass.fit(X_train, y_train)

###Predicting
y_pred_ica= ANN_ICAclass.predict(X_test)
y_expect_ica = y_test

##Accuracy
acc_dt1_ica = accuracy_score(y_expect_ica , y_pred_ica)
print(acc_dt1_ica)

### create confusion matrix
cm_dt1_ica= confusion_matrix(y_test , y_pred_ica)
print(cm_dt1_ica)

###ANN Classifier using RP

X_train,X_test, y_train, y_test = train_test_split(X_RP,Y,test_size = 0.25 , random_state = 0)

ANN_PCAclass= MLPClassifier(hidden_layer_sizes= (30,30,30,30),
                          activation='relu',  
                          batch_size='auto' ,
                          solver= 'sgd',
                          learning_rate= 'constant',
                          max_iter= 500,
                          random_state= 0 )
ANN_class.fit(X_train, y_train)

###Predicting
y_pred_rp= ANN_class.predict(X_test)
y_expect_rp = y_test

##Accuracy
acc_dt1_rp = accuracy_score(y_expect_rp, y_pred_rp)
print(acc_dt1_rp)

### create confusion matrix
cm_dt1_rp= confusion_matrix(y_test , y_pred_rp)
print(cm_dt1_rp)

###ANN Classifier using FS

X_train,X_test, y_train, y_test = train_test_split(X_FS,Y,test_size = 0.25 , random_state = 0)

ANN_FSclass= MLPClassifier(hidden_layer_sizes= (30,30,30,30),
                          activation='relu',  
                          batch_size='auto' ,
                          solver= 'sgd',
                          learning_rate= 'constant',
                          max_iter= 500,
                          random_state= 0 )
ANN_FSclass.fit(X_train, y_train)

###Predicting
y_pred_fs= ANN_FSclass.predict(X_test)
y_expect_fs = y_test

##Accuracy
acc_dt1_fs = accuracy_score(y_expect_fs, y_pred_fs)
print(acc_dt1_fs)

### create confusion matrix
cm_dt1_fs= confusion_matrix(y_test , y_pred_fs)
print(cm_dt1_fs)

###Plotting Accuraires

accuracies_energy = [acc_dt1 ,acc_dt1_pca,acc_dt1_rp,acc_dt1_ica,acc_dt1_fs]
accuracies_type = ['Normal','PCA','Random Project','ICA','Feature Selection']
plt.plot(accuracies_type , accuracies_energy , marker='o',color='blue')
plt.title(" Accuracies for different Dimension Reduction")
plt.xlabel("Reduction Techniques")
plt.grid()
plt.ylabel("Accuracy")
plt.show()

# EXPECTATION MAXIMIZATION

In [ ]:

bic = []
cv_types = ['spherical', 'tied', 'diag', 'full']
n_components = np.arange(1,11,1)
for i in cv_types:
    bic_cv = []
    for n in n_components:
        EM_Model = GMM(n_components=n, covariance_type=i, n_init=1)
        EM_Model.fit(X_FS)
        bic_cv.append(EM_Model.bic(X_FS)) ##Run same code by replacing X_FS,X_PCA,X_RP,X_ICA,X_s
    bic.append(bic_cv)
    
for i in range(0,len(cv_types)):
    plt.plot(n_components, bic[i], marker='o', label="Covariance Type = {}".format(cv_types[i]))
    plt.legend(fontsize='small')
    plt.grid()
    plt.xlabel('Number of Clusters', fontsize=14)
    plt.ylabel('BIC score', fontsize=14)
    plt.title('BIC score - Dataset 1', fontsize=14)


# ANN CLASSIFIER WITH OUTPUT CLASS LABEL

In [ ]:

X_train,X_test, y_train, y_test = train_test_split(y_kmeans ,Y,test_size = 0.25 , random_state = 0)
ANN_class= MLPClassifier(hidden_layer_sizes= (30,30,30,30),
                          activation='relu',  
                          batch_size='auto' ,
                          solver= 'sgd',
                          learning_rate= 'constant',
                          max_iter= 500,
                          random_state= 0 )
ANN_class.fit(X_train.reshape(-1,1), y_train)
#F
###Predicting
y_pred= ANN_class.predict(X_test.reshape(-1,1))
y_expect = y_test

##Accuracy
acc_dt1 = accuracy_score(y_expect , y_pred)
print(acc_dt1)

### create confusion matrix
cm_dt1= confusion_matrix(y_test , y_pred)
print(cm_dt1)

# DATASET2

In [ ]:
####Dataset2

df_data2=pd.read_csv("C:/Users/PC/Desktop/ML/Bank_Data.csv")
df_data2.head()

##Converting Variables to Categorical Data.

df_data2['default']=df_data2['default'].map({'unknown':0,'no':1,'yes':2})
df_data2['housing']=df_data2['housing'].map({'unknown':0,'no':1,'yes':2})
df_data2['loan']=df_data2['loan'].map({'unknown':0,'no':1,'yes':2})
df_data2['month']=df_data2['month'].map({'jan':1,'feb':2,'mar':3,'apr':4,'may':5,'jun':6,'jul':7,'aug':8,'sep':9,'oct':10,'nov':11,'dec':12})
df_data2['poutcome']=df_data2['poutcome'].map({'nonexistent':0,'failure':1,'success':2})
df_data=df_data2.drop(['duration'],axis=1)
#df_data2.head()

###Converting Nominal Variable.

collst=['job','marital','education','contact','day_of_week']
df_data2=pd.get_dummies(df_data2,columns=collst)

df_data2['y']=df_data2['y'].map({'yes':1,'no':0})

####Splitting Data into Train Test
X2=df_data2.drop('y',axis=1).values
Y2=df_data2['y'].values


####Splitting data into train test
X_train2,X_test2, y_train2, y_test2 = train_test_split(X2,Y2,test_size = 0.25 , random_state = 0)



# K-MEANS

In [ ]:
# Using elblow method to find optimal number of clusters
from sklearn.cluster import KMeans
wcss = []
for i in range (1,30):
    kmeans = KMeans(n_clusters=i ,  
                    init = "k-means++" , 
                    max_iter= 500 , 
                    n_init= 10 , 
                    random_state= 0 )
    kmeans.fit(X2)
    wcss.append(kmeans.inertia_)

plt.plot(range(1,30),wcss,marker='o')
plt.grid()
plt.title("Error Rate-Elbow Method")
plt.xlabel("Number of Cluster")
plt.ylabel("Error")
plt.show()

####K-MEANS
kmeans = KMeans(n_clusters= 4, 
                init = "k-means++" , 
                max_iter= 500 , 
                n_init= 10 , 
                random_state= 0)

y_kmeans = kmeans.fit_predict(X2)

# IMPLEMENTING DIMENSION REDUCTIONALITY PCA/ICA/RP/FS

In [ ]:
# Implementing PCA for Energy dataset to find maximum variance 

pca = PCA(n_components= None)
X2_PCA = pca.fit_transform(X2)
#X_test = pca.transform(X_test)
explained_variance = pca.explained_variance_ratio_

plt.plot(range(1,47),explained_variance,marker='o')
plt.title("PCA Variance")
plt.xlabel("Component Number")
plt.ylabel("Variance")
plt.grid()
plt.show()

pca_final_energy = PCA(n_components =2)
X2_PCA= pca_final_energy.fit_transform(X2)

wcss_pca = []
for i in range (1,20):
    kmeans = KMeans(n_clusters=i , 
                    init = "k-means++" , 
                    max_iter= 500 , 
                    n_init= 10 , 
                    random_state= 0 )
    kmeans.fit(X2_PCA)
    wcss_pca.append(kmeans.inertia_)

plt.plot(range(1,20),wcss_pca,marker='o')
plt.title("PCA-Elbow Method")
plt.grid()
plt.xlabel("Number of Cluster")
plt.ylabel("Error")
plt.show()

In [ ]:
from sklearn.decomposition import FastICA

ica = FastICA(tol = 1e-2, max_iter = 1000, n_components =2)
X2_ICA = ica.fit_transform(X2)

wcss_energy_ica = []
for i in range (1,21):
    kmeans = KMeans(n_clusters=i , 
                    init = "k-means++" , 
                    max_iter= 500 , 
                    n_init= 10 , 
                    random_state= 0 )
    kmeans.fit(X2_ICA)
    wcss_energy_ica.append(kmeans.inertia_)

plt.plot(range(1,21),wcss_energy_ica,marker='o')
plt.title("ICA-Elbow Method")
plt.grid()
plt.xlabel("Number of Cluster")
plt.ylabel("Error")
plt.show()


In [ ]:
from sklearn.random_projection import SparseRandomProjection
srp = SparseRandomProjection(eps = 0.1, n_components = 2)
X2_RP = srp.fit_transform(X2)

wcss_energy_ra = []
for i in range (1,11):
    kmeans = KMeans(n_clusters=i , 
                    init = "k-means++" , 
                    max_iter= 500 , 
                    n_init= 10 , 
                    random_state= 0 )
    kmeans.fit(X2_RP)
    wcss_energy_ra.append(kmeans.inertia_)

plt.plot(range(1,11),wcss_energy_ra,marker='o')
plt.title("Randomized Projection-Elbow Method")
plt.grid()
plt.xlabel("Number of Cluster")
plt.ylabel("Error")
plt.show()

In [ ]:
####Feature Selection


# Using RFE to do the feature selection by Random Forest
from sklearn.feature_selection import RFECV
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc

dt = DecisionTreeClassifier(criterion = 'entropy', max_depth=6)
rfecv = RFECV(estimator=dt, step=1, cv=5, n_jobs=-1,
              scoring='accuracy')
rfecv.fit(X2,Y2)

print("Optimal number of features : %d" % rfecv.n_features_)
print(rfecv.ranking_)


plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score")
plt.grid()
plt.title("Feature scores")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_,marker='o')
plt.show()


X2_FS = X2[:,(2,5)]

wcss_FS = []
for i in range (1,20):
    kmeans = KMeans(n_clusters=i , 
                    init = "k-means++" , 
                    max_iter= 500 , 
                    n_init= 10 , 
                    random_state= 0 )
    kmeans.fit(X2_FS)
    wcss_FS.append(kmeans.inertia_)

plt.plot(range(1,20),wcss_FS,marker='o')
plt.title("Feature Selection-Elbow Method ")
plt.grid()
plt.xlabel("Number of Cluster")
plt.ylabel("Error")
plt.show()

# IMPLEMENTING ANN WITH DIMENTIONALIY REDUCTION

In [ ]:
###ANN

####FINAL CLASSIFIER A-NN####

X_train2,X_test2, y_train2, y_test2 = train_test_split(X2,Y2,test_size = 0.25 , random_state = 0)

ANN_class= MLPClassifier(hidden_layer_sizes= (30,30,30,30),
                          activation='relu',  
                          batch_size='auto' ,
                          solver= 'adam',
                          learning_rate= 'constant',
                          max_iter= 300,
                          random_state= 0 )
ANN_class.fit(X_train2, y_train2)

##Making Prediction
y2_pred = ANN_class.predict(X_test2)
y2_expect = y_test2

### Accuracy
acc_dt2 = accuracy_score(y2_expect , y2_pred)
print(acc_dt2)

##Confusion matrix
cm_dt2 = confusion_matrix(y_test2 , y2_pred)
print(cm_dt2)

###PCA
X_train2,X_test2, y_train2, y_test2 = train_test_split(X2_PCA,Y2,test_size = 0.25 , random_state = 0)

ANN_PCAclass= MLPClassifier(hidden_layer_sizes= (30,30,30,30),
                          activation='relu',  
                          batch_size='auto' ,
                          solver= 'adam',
                          learning_rate= 'constant',
                          max_iter= 300,
                          random_state= 0 )
ANN_PCAclass.fit(X_train2, y_train2)

##Making Prediction
y2_pred = ANN_PCAclass.predict(X_test2)
y2_expect = y_test2

### Accuracy
acc_dt2_pca = accuracy_score(y2_expect , y2_pred)
print(acc_dt2_pca)

##Confusion matrix
cm_dt2_pca = confusion_matrix(y_test2 , y2_pred)
print(cm_dt2_pca)

##ICA
X_train2,X_test2, y_train2, y_test2 = train_test_split(X2_ICA,Y2,test_size = 0.25 , random_state = 0)

ANN_ICAclass= MLPClassifier(hidden_layer_sizes= (30,30,30,30),
                          activation='relu',  
                          batch_size='auto' ,
                          solver= 'adam',
                          learning_rate= 'constant',
                          max_iter= 300,
                          random_state= 0 )
ANN_ICAclass.fit(X_train2, y_train2)

##Making Prediction
y2_pred = ANN_ICAclass.predict(X_test2)
y2_expect = y_test2

### Accuracy
acc_dt2_ica = accuracy_score(y2_expect , y2_pred)
print(acc_dt2_ica)

##Confusion matrix
cm_dt2_ica = confusion_matrix(y_test2 , y2_pred)
print(cm_dt2_ica)

####FINAL CLASSIFIER A-NN-RP####

X_train2,X_test2, y_train2, y_test2 = train_test_split(X2_RP,Y2,test_size = 0.25 , random_state = 0)

ANN_RPclass= MLPClassifier(hidden_layer_sizes= (30,30,30,30),
                          activation='relu',  
                          batch_size='auto' ,
                          solver= 'adam',
                          learning_rate= 'constant',
                          max_iter= 300,
                          random_state= 0 )
ANN_RPclass.fit(X_train2, y_train2)

##Making Prediction
y2_pred = ANN_RPclass.predict(X_test2)
y2_expect = y_test2

### Accuracy
acc_dt2_rp = accuracy_score(y2_expect , y2_pred)
print(acc_dt2_rp)

##Confusion matrix
cm_dt2_rp = confusion_matrix(y_test2 , y2_pred)
print(cm_dt2_rp)

####FS
X_train2,X_test2, y_train2, y_test2 = train_test_split(X2_FS,Y2,test_size = 0.25 , random_state = 0)

ANN_FSclass= MLPClassifier(hidden_layer_sizes= (30,30,30,30),
                          activation='relu',  
                          batch_size='auto' ,
                          solver= 'adam',
                          learning_rate= 'constant',
                          max_iter= 300,
                          random_state= 0 )
ANN_FSclass.fit(X_train2, y_train2)

##Making Prediction
y2_pred = ANN_FSclass.predict(X_test2)
y2_expect = y_test2

### Accuracy
acc_dt2_fs = accuracy_score(y2_expect , y2_pred)
print(acc_dt2_fs)

##Confusion matrix
cm_dt2_fs = confusion_matrix(y_test2 , y2_pred)
print(cm_dt2_fs)

###Plotting Accuraires

accuracies_energy = [acc_dt2 ,acc_dt2_pca,acc_dt2_rp,acc_dt2_ica,acc_dt2_fs]
accuracies_type = ['Normal','PCA','Random Project','ICA','Feature Selection']
plt.plot(accuracies_type , accuracies_energy , marker='o',color='blue')
plt.title(" Accuracies for different Dimension Reduction")
plt.xlabel("Reduction Techniques")
plt.grid()
plt.ylabel("Accuracy")
plt.show()

# EXPECTATION MAXIMIZATION

In [ ]:
bic = []
cv_types = ['spherical', 'tied', 'diag', 'full']
n_components = np.arange(1,11,1)
for i in cv_types:
    bic_cv = []
    for n in n_components:
        EM_Model = GMM(n_components=n, covariance_type=i, n_init=2)
        EM_Model.fit(X2)
        bic_cv.append(EM_Model.bic(X2))
    bic.append(bic_cv)
   
##Plotting
for i in range(0,len(cv_types)):
    plt.plot(n_components, bic[i], marker='o', label="Covariance Type = {}".format(cv_types[i]))
    plt.legend(fontsize='small')
    plt.grid()
    plt.xlabel('Number of Clusters', fontsize=14)
    plt.ylabel('BIC score', fontsize=14)
    plt.title('BIC score - Dataset 2', fontsize=14)

    

# ANN CLASSIFIER WITH OUTPUT LABEL

In [ ]:
X_train2,X_test2, y_train2, y_test2 = train_test_split(y_kmeans,Y2,test_size = 0.25 , random_state = 0)
ANN_class= MLPClassifier(hidden_layer_sizes= (30,30,30,30),
                          activation='relu',  
                          batch_size='auto' ,
                          solver= 'sgd',
                          learning_rate= 'constant',
                          max_iter= 500,
                          random_state= 0 )
ANN_class.fit(X_train2.reshape(-1,1), y_train2)
#F
###Predicting
y_pred= ANN_class.predict(X_test2.reshape(-1,1))
y_expect = y_test2

##Accuracy
acc_dt1 = accuracy_score(y_expect , y_pred)
print(acc_dt1)

### create confusion matrix
cm_dt1= confusion_matrix(y_test2 , y_pred)
print(cm_dt1)